In [ ]:
# install langchain_google_genai library
# !pip install langchain_google_genai
# !pip install python-dotenv

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
#check that the environment variable GOOGLE_API_KEY is set
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("GOOGLE_API_KEY environment variable is not set.") 
else:
    api_key=os.environ["GOOGLE_API_KEY"],
    
#print(f"GOOGLE_API_KEY is set: {api_key}")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

In [24]:
# --- Basic usage (single prompt) ---
print("--- Basic LLM Usage ---")
result = llm.invoke("What is the primary function of a CPU?")
print(result.content)

--- Basic LLM Usage ---
The primary function of a CPU (Central Processing Unit) is to **fetch, decode, and execute instructions**.  It's essentially the "brain" of the computer, carrying out the calculations and logical operations that make the computer work.  This cycle of fetching, decoding, and executing is continuous, allowing the computer to perform a wide variety of tasks.


In [28]:
# --- Using with ChatPromptTemplate and Chains (recommended for structured interactions) ---
print("\n--- Using with ChatPromptTemplate and Chains ---")

# 1. Define your prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer concisely."),
    ("human", "{question}")  # <-- Tuple format for templated message
])


--- Using with ChatPromptTemplate and Chains ---


In [29]:
print("Prompt Template:")
print(prompt)

Prompt Template:
input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant. Answer concisely.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [30]:
question_to_test = "What is the primary function of a CPU?"

print("--- Checking Prompt Rendering ---")
rendered_messages = prompt.invoke({"question": question_to_test})

print("Rendered Messages:")
print(f"Type of rendered_messages: {type(rendered_messages)}") # Should be <class 'langchain_core.messages.BaseMessages'>
print("Messages in rendered prompt:")
for msg in rendered_messages.messages:
    print(f"- {msg.type.capitalize()} Message: {msg.content}")

--- Checking Prompt Rendering ---
Rendered Messages:
Type of rendered_messages: <class 'langchain_core.prompt_values.ChatPromptValue'>
Messages in rendered prompt:
- System Message: You are a helpful AI assistant. Answer concisely.
- Human Message: What is the primary function of a CPU?


In [32]:
# 2. Define an output parser (optional, but good practice for structured output)
output_parser = StrOutputParser()

# 3. Create a chain
chain = prompt | llm | output_parser

# 4. Invoke the chain
question = "What is the primary function of a CPU, elaborate mode in term of interview question?"
answer = chain.invoke({"question": question})
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is the primary function of a CPU, elaborate mode in term of interview question?
Answer: The CPU (Central Processing Unit) is the primary component of a computer that executes instructions of a computer program.  It fetches instructions from memory, decodes them, performs the specified operations (arithmetic, logical, control), and stores the results back in memory.  Think of it as the "brain" of the computer, responsible for carrying out almost all the tasks the computer performs.


In [36]:
# --- Example with a more complex prompt and few-shot examples ---
print("\n--- Example with Few-Shot Prompting ---")
few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert sentiment analyzer. Classify the sentiment of the following texts as 'Positive', 'Negative', or 'Neutral'."),
    ("human", "This movie was fantastic!"),
    ("human", "Positive"),
    ("human", "The service was okay, nothing special."),
    ("human", "Neutral"),
    ("human", "I absolutely hated the food."),
    ("human", "Negative"),
    ("human", "{text_to_classify}")  # <-- Placeholder for the text to classify
])

sentiment_chain = few_shot_prompt | llm | output_parser

text_to_classify = "The weather today is surprisingly superhot and humid and I can't take it."
sentiment = sentiment_chain.invoke({"text_to_classify": text_to_classify})
print(f"Text: {text_to_classify}")
print(f"Sentiment:Sentiment: {sentiment}")


--- Example with Few-Shot Prompting ---
Text: The weather today is surprisingly superhot and humid and I can't take it.
Sentiment:Sentiment: Negative


In [37]:
text_to_classify = "The weather today is surprisingly pleasant."
sentiment = sentiment_chain.invoke({"text_to_classify": text_to_classify})
print(f"Text: {text_to_classify}")
print(f"Sentiment:Sentiment: {sentiment}")

Text: The weather today is surprisingly pleasant.
Sentiment:Sentiment: Positive
